In [1]:
!wget "https://media.githubusercontent.com/media/pradeep-016/ML_Course/refs/heads/main/7)%20Recommendation%20Systems/Hybrid%20Recommendation%20Systems/tmdb_5000_movies.csv"
!wget "https://media.githubusercontent.com/media/pradeep-016/ML_Course/refs/heads/main/7)%20Recommendation%20Systems/Hybrid%20Recommendation%20Systems/tmdb_5000_credits.csv"

--2024-12-08 14:29:12--  https://media.githubusercontent.com/media/pradeep-016/ML_Course/refs/heads/main/7)%20Recommendation%20Systems/Hybrid%20Recommendation%20Systems/tmdb_5000_movies.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5698602 (5.4M) [text/plain]
Saving to: ‘tmdb_5000_movies.csv’

tmdb_5000_movies.cs 100%[===================>]   5.43M  --.-KB/s    in 0.04s   

2024-12-08 14:29:13 (144 MB/s) - ‘tmdb_5000_movies.csv’ saved [5698602/5698602]

--2024-12-08 14:29:13--  https://media.githubusercontent.com/media/pradeep-016/ML_Course/refs/heads/main/7)%20Recommendation%20Systems/Hybrid%20Recommendation%20Systems/tmdb_5000_credits.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 1

In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

In [3]:
credits_df = pd.read_csv('tmdb_5000_credits.csv')
movies_df = pd.read_csv('tmdb_5000_movies.csv')

In [4]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [5]:
credits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [6]:
merge_df = pd.merge(movies_df, credits_df, left_on = 'id', right_on = 'movie_id')

In [7]:
merge_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [8]:
def extract_names(json_str):
  try:
    return " ".join([entry['name'] for entry in literal_eval(json_str)])
  except:
    return ""

In [9]:
merge_df['genres'] = merge_df['genres'].apply(extract_names)
merge_df['keywords'] = merge_df['keywords'].apply(extract_names)

In [10]:
merge_df['genres'].head()

,genres
0,Action Adventure Fantasy Science Fiction
1,Adventure Fantasy Action
2,Action Adventure Crime
3,Action Crime Drama Thriller
4,Action Adventure Science Fiction


In [11]:
merge_df['combined_features'] = merge_df['overview'].fillna("")+" "+merge_df['genres'].fillna(" ")+ " "+ merge_df['keywords'].fillna(" ")

In [12]:
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(merge_df['combined_features'])

In [13]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [14]:
ratings_data = {
    'user_id': [1, 2, 3, 1, 2, 3],
    'movie_id': [1, 1, 2, 2, 3, 3],
    'rating': [5, 4, 3, 2, 5, 1]
}

In [15]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [16]:
ratings = pd.DataFrame(ratings_data)

In [17]:
rating_matrix = ratings.pivot(index = 'user_id', columns = 'movie_id', values = 'rating').fillna(0)

In [18]:
rating_sparse = csr_matrix(rating_matrix.values)

In [19]:
n_components = min(rating_sparse.shape[0], rating_sparse.shape[1])

In [20]:
svd = TruncatedSVD(n_components = n_components)
svd_matrix = svd.fit_transform(rating_sparse)

In [21]:
def content_based_recommendations(movie_title, cosine_sim = cosine_sim, top_n = 10):
  idx = merge_df.index[merge_df['title_x'] == movie_title].tolist()[0]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
  sim_scores = sim_scores[1:top_n+1]
  movie_indices = [i[0] for i in sim_scores]
  return merge_df['title_x'].iloc[movie_indices]

In [22]:
def collaborative_filtering(user_id, svd_matrix = svd_matrix, rating_matrix = rating_matrix, top_n = 10):
  scores = svd_matrix[user_id - 1, :]
  movie_indices = np.argsort(scores)[-top_n:][::-1]
  movie_ids = rating_matrix.columns[movie_indices].tolist()
  movie_titles = merge_df.loc[merge_df['id'].isin(movie_ids), 'title_x'].tolist()
  return movie_titles

In [23]:
def hybrid_recommendations(user_id, title, top_n = 10):
  content_recs = content_based_recommendations(title, top_n = top_n)
  collab_recs = collaborative_filtering(user_id,svd_matrix, rating_matrix, top_n)
  combined_recs = list(set(content_recs).union(set(collab_recs)))
  return combined_recs[:top_n]

In [24]:
user_id = 1
movie_title = "Avatar"
top_n = 10
recommendations = hybrid_recommendations(user_id, movie_title, top_n=top_n)

print(f"Hybrid Recommendations for User {user_id} based on Movie '{movie_title}':")
for idx, rec in enumerate(recommendations, 1):
    print(f"{idx}. {rec}")

Hybrid Recommendations for User 1 based on Movie 'Avatar':
1. Moonraker
2. Alien³
3. Aliens
4. Lifeforce
5. Spaceballs
6. Mission to Mars
7. Alien
8. Lockout
9. Treasure Planet
10. Planet of the Apes
